In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
import random
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# Logic Bot


In [ ]:
data_collection = []


class MineSweeperLogicBot():
    def __init__(self, width, height, num_mines):
        self.width = width
        self.height = height
        self.num_mines = num_mines
        self.new_board()


    def new_board(self):
        self.board = [[-1 for _ in range(self.width)] for _ in range(self.height)]
        self.cells_remaining = set((x, y) for x in range(self.height) for y in range(self.width))
        self.mine_cord = []
        self.inferred_safe = set()
        self.inferred_mine = set()
        self.clue_numbers = {}
        self.mine_mask = [[0 for _ in range(self.width)] for _ in range(self.height)]

    def get_information(self, cell_cord):
        num_inferred_mines, num_inferred_safe = 0, 0
        unrevealed_neighbors = []
        neighbours = self.get_neighbours(cell_cord)

        for cord in neighbours:
            if cord in self.inferred_mine:
                num_inferred_mines += 1
            if cord in self.inferred_safe or cord in self.clue_numbers:
                num_inferred_safe += 1
            if cord in self.cells_remaining and cord not in self.clue_numbers:
                unrevealed_neighbors.append(cord)

        return num_inferred_mines, num_inferred_safe, unrevealed_neighbors, len(neighbours)

    def get_neighbours(self, cell_cord):
        x, y = cell_cord
        neighbours = []
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0:
                    continue
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.height and 0 <= ny < self.width:
                    neighbours.append((nx, ny))
        return neighbours

    def get_clue(self, cell_cord):
        clue = 0
        neighbours = self.get_neighbours(cell_cord)
        for cord in neighbours:
            if cord in self.mine_cord:
                clue += 1
            else:
                self.mine_mask[cord[0]][cord[1]] = 1

        return clue


    def update_inferences(self, picked_cell, clue):
        num_inferred_mines, num_inferred_safe, unrevealed_neighbors, num_neighbours = self.get_information(picked_cell)

        # finding mine spots
        if clue - num_inferred_mines == len(unrevealed_neighbors):
            self.inferred_mine.update(unrevealed_neighbors)
            self.cells_remaining.difference_update(unrevealed_neighbors)

        # finding safe spots
        if num_neighbours - clue - num_inferred_safe == len(unrevealed_neighbors):
            self.inferred_safe.update(unrevealed_neighbors)
            self.cells_remaining.difference_update(unrevealed_neighbors)


    def record_state(self, board, mine_mask):
        # self.i += 1
        # print(self.i)
        # Flatten the board and mine_mask for simpler tensor conversion
        # flattened_board = [item for sublist in board for item in sublist]
        # flattened_mine_mask = [item for sublist in self.mine_mask for item in sublist]
        data_collection.append((board, mine_mask))

    def play(self):
        first_turn = True
        # self.mine_mask = [[1]*self.width for _ in range(self.height)]

        while self.cells_remaining:
            if first_turn:
                picked_cell = random.choice(list(self.cells_remaining))
                safe_start = set(self.get_neighbours(picked_cell)) | {picked_cell}
                self.mine_cord = random.sample(list(self.cells_remaining - safe_start), self.num_mines)
                # for mine in self.mine_cord:
                #     self.mine_mask[mine[0]][mine[1]] = 0
                first_turn = False
            elif self.inferred_safe:
                picked_cell = self.inferred_safe.pop()
            else:
                picked_cell = random.choice(list(self.cells_remaining))

            self.cells_remaining.discard(picked_cell)
            if picked_cell in self.mine_cord:
                # self.board[picked_cell[0]][picked_cell[1]] = 9
                self.record_state(self.board, self.mine_mask)
                return False  # Game over, hit a mine

            clue = self.get_clue(picked_cell)
            self.board[picked_cell[0]][picked_cell[1]] = clue
            self.clue_numbers[picked_cell] = clue
            self.record_state(self.board, self.mine_mask)
            self.update_inferences(picked_cell, clue)

            if self.inferred_mine == self.num_mines and all(self.board[x][y] != -1 for x in range(self.height) for y in range(self.width) if (x, y) not in self.mine_cord):
                return True

        return True


# Model Training

In [ ]:

def train_and_test_data_lstm(data_path):
    # Load the saved data
    data = torch.load(data_path)

    # Flatten the widthxheight grids into sequences of w*h features
    states, labels = zip(*data)
    states = torch.stack([torch.tensor(state, dtype=torch.float32).view(1, -1) for state in states])
    labels = torch.stack([torch.tensor(label, dtype=torch.float32).flatten()  for label in labels])

    # Splitting and DataLoader setup remains the same
    num_train = int(0.8 * len(states))
    train_states, test_states = states[:num_train], states[num_train:]
    train_labels, test_labels = labels[:num_train], labels[num_train:]

    train_dataset = torch.utils.data.TensorDataset(train_states, train_labels)
    test_dataset = torch.utils.data.TensorDataset(test_states, test_labels)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    return train_loader, test_loader


In [ ]:
class MinesweeperLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MinesweeperLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initial hidden and cell states
        h0 = torch.zeros(1, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_dim).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Decode the hidden state of the last time step
        out = self.linear(out[:, -1, :])
        return out

In [ ]:
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target.float())
            total_loss += loss.item()
    return total_loss / len(test_loader)

In [ ]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f'Model saved to {path}')

# Initialize and train the model
def start_train(input_dim, hidden_dim, output_dim, model_path,data_path):
    # train_loader, test_loader =  train_and_test_data()
    # board_width = 9
    # board_height = 9
    # model = MinesweeperCNN(board_width, board_height)
    # train_losses, test_losses = train_and_evaluate(model, train_loader, test_loader)

    train_loader, test_loader = train_and_test_data_lstm(data_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MinesweeperLSTM(input_dim, hidden_dim, output_dim).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    num_epochs = 10
    for epoch in range(num_epochs):
        train_loss = train_model(model, train_loader, criterion, optimizer, device)
        test_loss = evaluate_model(model, test_loader, criterion, device)
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

    save_model(model, model_path)

# Testing Trained Model

In [ ]:
def load_model(input_dim, hidden_dim, output_dim, path):
    # model = MinesweeperCNN(9,9)  # Initialize the model structure as defined during training
    model = MinesweeperLSTM(input_dim, hidden_dim, output_dim)
    # model = MinesweeperLSTM(81, 128, 81)
    model.load_state_dict(torch.load(path))
    # model.eval()  # Set the model to evaluation mode
    return model

In [ ]:
def find_safest_spot(model, board):
    # Convert the board to the input tensor format expected by the model
    width = len(board)
    height = len(board[0])
    input_tensor = torch.tensor(board, dtype=torch.float32).view(1, 1, height, width)
    input_tensor = input_tensor.flatten(start_dim=2)  # Flatten

    # Get the model's prediction
    model.eval()
    with torch.no_grad():
        output = model(input_tensor).view(height, width)  # Reshape back to actual grid

    # Find the safest spot: highest probability of being safe but still unrevealed (-1)
    max_prob = 0
    safest_spot = None
    for i in range(height):
        for j in range(width):
            if board[i][j] == -1 and output[i][j] > max_prob:  # Check for unrevealed and high probability
                max_prob = output[i][j]
                safest_spot = (i, j)

    return safest_spot

In [ ]:
# game = MineSweeperLogicBot(width,height,num_mines)


def model_play(model, game):
    first_turn = True

    while game.cells_remaining:
        if first_turn:
            picked_cell = random.choice(list(game.cells_remaining))
            safe_start = set(game.get_neighbours(picked_cell)) | {picked_cell}
            game.mine_cord = set(random.sample(list(game.cells_remaining - safe_start), game.num_mines))
            first_turn = False
        else:
            picked_cell = find_safest_spot(model, game.board)
            if picked_cell == None:
                return RuntimeError


        game.cells_remaining.discard(picked_cell)

        if picked_cell in game.mine_cord:
            # print(f"Move at {picked_cell} , mine hit")
            # for row in game.board:
            #     print(' '.join(str(x) for x in row))
            return False  # Game over, hit a mine

        clue = game.get_clue(picked_cell)
        game.board[picked_cell[0]][picked_cell[1]] = clue

        # print(f"Move at {picked_cell} with clue {clue}")
        # for row in game.board:
        #     print(' '.join(str(x) for x in row))



        if all(game.board[x][y] != -1 for x in range(game.height) for y in range(game.width) if (x, y) not in game.mine_cord):
            return True

# EASY LEVEL

In [1]:
# GENERATING DATA

width = 9
height = 9
num_mines = 10

data_path = "easy_minesweeper_data.pt"


def main():
    game = MineSweeperLogicBot(width,height,num_mines)



    won, lost = 0,0

    for i in range(10000):
        game.new_board()

        result = game.play()

        if result:
            # print(f"Won at {i}th game!\n")
            won += 1
        else:
            # print(f"Lost at {i}th game...\n---------------\n")
            lost += 1

    print("\nLogic Bot:\n")
    print(f"Total game WON ===> {won}\n")
    print(f"Total game Lost ===> {lost}\n")

    print("The data len :", len(data_collection))

    torch.save(data_collection, data_path)
    print("Saved the dataset!\n")


main()

Logic Bot:

Total game WON ===> 494

Total game Lost ===> 9506

The data len : 545906
Saved the dataset!


In [2]:
# MODEL TRAINING

model_path = "easy_minesweeper_model.pth"
input_dim = width * height
hidden_dim= 2 *input_dim
output_dim= input_dim

start_train(input_dim, hidden_dim, output_dim, model_path, data_path)




Epoch 1, Train Loss: 0.6295, Test Loss: 0.5599
Epoch 2, Train Loss: 0.5024, Test Loss: 0.4697
Epoch 3, Train Loss: 0.4592, Test Loss: 0.4567
Epoch 4, Train Loss: 0.4525, Test Loss: 0.4527
Epoch 5, Train Loss: 0.4486, Test Loss: 0.4488
Epoch 6, Train Loss: 0.4444, Test Loss: 0.4443
Epoch 7, Train Loss: 0.4396, Test Loss: 0.4391
Epoch 8, Train Loss: 0.4342, Test Loss: 0.4336
Epoch 9, Train Loss: 0.4285, Test Loss: 0.4278
Epoch 10, Train Loss: 0.4228, Test Loss: 0.4222
Model saved to easy_minesweeper_model.pth


In [3]:
# COMPARISON

l_won, l_lost, m_won, m_lost = 0,0,0,0

game1 = MineSweeperLogicBot(width,height,num_mines)


for i in range(1000):
    game1.new_board()

    result = game1.play()

    if result:
        # print(f"Won at {i}th game!\n")
        l_won += 1
    else:
        # print(f"Lost at {i}th game...\n---------------\n")
        l_lost += 1

print("\nLogic Bot:\n")
print(f"Total game WON ===> {l_won}\n")
print(f"Total game Lost ===> {l_lost}\n")






game2 = MineSweeperLogicBot(width,height,num_mines)

for i in range(1000):
    game2.new_board()


    result = model_play(load_model(input_dim, hidden_dim, output_dim, model_path), game2)

    if result:
        # print(f"Won at {i}th game!\n")
        m_won += 1
    else:
        # print(f"Lost at {i}th game...\n---------------\n")
        m_lost += 1
print("\nTrained Model:\n")

print(f"Total game WON ===> {m_won}\n")
print(f"Total game Lost ===> {m_lost}\n")


Logic Bot:

Total game WON ===> 53

Total game Lost ===> 947


Trained Model:

Total game WON ===> 1000

Total game Lost ===> 0


# INTERMEDIATE LEVEL

In [4]:
# GENERATING DATA

width = 16
height = 16
num_mines = 40

data_path = "intermediate_minesweeper_data.pt"


def main():
    game = MineSweeperLogicBot(width,height,num_mines)



    won, lost = 0,0

    for i in range(10000):
        game.new_board()

        result = game.play()

        if result:
            # print(f"Won at {i}th game!\n")
            won += 1
        else:
            # print(f"Lost at {i}th game...\n---------------\n")
            lost += 1

    print("\nLogic Bot:\n")
    print(f"Total game WON ===> {won}\n")
    print(f"Total game Lost ===> {lost}\n")

    print("The data len :", len(data_collection))

    torch.save(data_collection, data_path)
    print("Saved the dataset!\n")


main()



Logic Bot:

Total game WON ===> 0

Total game Lost ===> 10000

The data len : 951011
Saved the dataset!


In [5]:
# MODEL TRAINING

model_path = "intermediate_minesweeper_model.pth"
input_dim = width * height
hidden_dim= 2 *input_dim
output_dim= input_dim

start_train(input_dim, hidden_dim, output_dim, model_path,data_path)



Epoch 1, Train Loss: 0.6765, Test Loss: 0.6552
Epoch 2, Train Loss: 0.6221, Test Loss: 0.5850
Epoch 3, Train Loss: 0.5529, Test Loss: 0.5243
Epoch 4, Train Loss: 0.5087, Test Loss: 0.4934
Epoch 5, Train Loss: 0.4859, Test Loss: 0.4756
Epoch 6, Train Loss: 0.4706, Test Loss: 0.4621
Epoch 7, Train Loss: 0.4582, Test Loss: 0.4512
Epoch 8, Train Loss: 0.4483, Test Loss: 0.4426
Epoch 9, Train Loss: 0.4404, Test Loss: 0.4358
Epoch 10, Train Loss: 0.4340, Test Loss: 0.4303
Model saved to intermediate_minesweeper_model.pth


In [6]:
# COMPARISON

l_won, l_lost, m_won, m_lost = 0,0,0,0

game1 = MineSweeperLogicBot(width,height,num_mines)


for i in range(1000):
    game1.new_board()

    result = game1.play()

    if result:
        # print(f"Won at {i}th game!\n")
        l_won += 1
    else:
        # print(f"Lost at {i}th game...\n---------------\n")
        l_lost += 1

print("\nLogic Bot:\n")
print(f"Total game WON ===> {l_won}\n")
print(f"Total game Lost ===> {l_lost}\n")





game2 = MineSweeperLogicBot(width,height,num_mines)

for i in range(1000):
    game2.new_board()


    result = model_play(load_model(input_dim, hidden_dim, output_dim, model_path), game2)

    if result:
        # print(f"Won at {i}th game!\n")
        m_won += 1
    else:
        # print(f"Lost at {i}th game...\n---------------\n")
        m_lost += 1
print("\nTrained Model:\n")

print(f"Total game WON ===> {m_won}\n")
print(f"Total game Lost ===> {m_lost}\n")

Logic Bot:

Total game WON ===> 0

Total game Lost ===> 1000


Trained Model:

Total game WON ===> 1000

Total game Lost ===> 0


# EXPERT LEVEL

In [7]:
# GENERATING DATA

width = 30
height = 16
num_mines = 99

data_path = "expert_minesweeper_data.pt"


def main():
    game = MineSweeperLogicBot(width,height,num_mines)



    won, lost = 0,0

    for i in range(10000):
        game.new_board()

        result = game.play()

        if result:
            # print(f"Won at {i}th game!\n")
            won += 1
        else:
            # print(f"Lost at {i}th game...\n---------------\n")
            lost += 1

    print("\nLogic Bot:\n")
    print(f"Total game WON ===> {won}\n")
    print(f"Total game Lost ===> {lost}\n")

    print("The data len :", len(data_collection))

    torch.save(data_collection, data_path)
    print("Saved the dataset!\n")


main()


Logic Bot:

Total game WON ===> 0

Total game Lost ===> 10000

The data len : 612994
Saved the dataset!


In [8]:
# MODEL TRAINING

model_path = "expert_minesweeper_model.pth"
input_dim = width * height
hidden_dim= 2 *input_dim
output_dim= input_dim

start_train(input_dim, hidden_dim, output_dim, model_path,data_path)



Epoch 1, Train Loss: 0.6266, Test Loss: 0.5542
Epoch 2, Train Loss: 0.5397, Test Loss: 0.5265
Epoch 3, Train Loss: 0.5136, Test Loss: 0.4944
Epoch 4, Train Loss: 0.4762, Test Loss: 0.4560
Epoch 5, Train Loss: 0.4401, Test Loss: 0.4232
Epoch 6, Train Loss: 0.4108, Test Loss: 0.3982
Epoch 7, Train Loss: 0.3893, Test Loss: 0.3805
Epoch 8, Train Loss: 0.3734, Test Loss: 0.3671
Epoch 9, Train Loss: 0.3608, Test Loss: 0.3562
Epoch 10, Train Loss: 0.3504, Test Loss: 0.3472
Model saved to expert_minesweeper_model.pth


In [9]:
# COMPARISON

l_won, l_lost, m_won, m_lost = 0,0,0,0

game1 = MineSweeperLogicBot(width,height,num_mines)


for i in range(1000):
    game1.new_board()

    result = game1.play()

    if result:
        # print(f"Won at {i}th game!\n")
        l_won += 1
    else:
        # print(f"Lost at {i}th game...\n---------------\n")
        l_lost += 1

print("\nLogic Bot:\n")
print(f"Total game WON ===> {l_won}\n")
print(f"Total game Lost ===> {l_lost}\n")





game2 = MineSweeperLogicBot(width,height,num_mines)

for i in range(1000):
    game2.new_board()


    result = model_play(load_model(input_dim, hidden_dim, output_dim, model_path), game2)

    if result:
        # print(f"Won at {i}th game!\n")
        m_won += 1
    else:
        # print(f"Lost at {i}th game...\n---------------\n")
        m_lost += 1
print("\nTrained Model:\n")

print(f"Total game WON ===> {m_won}\n")
print(f"Total game Lost ===> {m_lost}\n")



Logic Bot:

Total game WON ===> 0

Total game Lost ===> 1000


Trained Model:

Total game WON ===> 1000

Total game Lost ===> 0
